# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
file_path = Path('../Starter_Code/Data/neighborhoods_coordinates.csv')
hood_coor = pd.read_csv(file_path)


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year(sfo_data):
    """Housing Units Per Year."""
    housing_units = sfo_data.groupby('year').mean()
    housing_units = housing_units.drop(["sale_price_sqr_foot", "gross_rent"], axis=1)
    ave_min = housing_units.min()
    ave_max = housing_units.max()
    ave_std = housing_units.std()
    housing_plot = housing_units.hvplot.bar(ylim=[ave_min[0],ave_max[0]]).opts(bgcolor='white')
    return housing_plot

def average_gross_rent(sfo_data):
    """Average Gross Rent in San Francisco Per Year."""
    gross_rent = sfo_data.groupby('year').mean()
    gross_rent = gross_rent.drop(["sale_price_sqr_foot", "housing_units"], axis=1)
    gross_plot = gross_rent.hvplot.line()
    return gross_plot

def average_sales_price(sfo_data):
    """Average Sales Price Per Year."""
    sqr_foot = sfo_data.groupby('year').mean()
    sqr_foot = sqr_foot.drop(["housing_units", "gross_rent"], axis=1)
    sqr_plot = sqr_foot.hvplot.line()
    return sqr_plot

def average_price_by_neighborhood(sfo_data):
    """Average Prices by Neighborhood."""
    hood_year = sfo_data.groupby(['year', 'neighborhood']).mean()
    hood_year_plot = hood_year['sale_price_sqr_foot'].hvplot.bar(
    title='Average Price Per Square Foot',
    xlabel= "Year",
    ylabel="housing_units",
    groupby="neighborhood")
    return hood_year_plot

def top_most_expensive_neighborhoods(sfo_data):
    """Top 10 Most Expensive Neighborhoods."""
    exp_hoods = sfo_data.groupby(["neighborhood"]).mean()
    ten_exp_hood = exp_hoods.sort_values('sale_price_sqr_foot', ascending=True).tail(10)
    ten_exp_plot = ten_exp_hood.drop(['housing_units', 'gross_rent'], axis=1)
    plot = ten_exp_plot.hvplot(kind='bar', rot=90)
    return plot

def most_expensive_neighborhoods_rent_sales(sfo_data):
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""
    mean_values = sfo_data.groupby(by='neighborhood').mean()
    mean_values = mean_values.sort_values('sale_price_sqr_foot', ascending=False).reset_index()
    hood_locations = pd.concat([hood_coor, mean_values], axis='columns', join='inner')
    hood_map = px.scatter_mapbox(hood_locations,
    lat='Lat',
    lon='Lon',
    color='gross_rent',
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=10,
    zoom=10,
    hover_name="Neighborhood",
    title="average sale price per square foot and gross rent in San Francisco")
    hood_map
    return hood_map
def parallel_coordinates():
    """Parallel Coordinates Plot."""




def parallel_categories():
    """Parallel Categories Plot."""
    



def neighborhood_map():
    """Neighborhood Map."""



def sunburst():
    """Sunburst Plot."""
    


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard 

# Create a tab layout for the dashboard make sure to call the function 
Housing_Units = pn.Column("## Housing Units Per Year", housing_units_per_year(sfo_data), background)
Gross_Rent = pn.Column("# Average Gross Rent Per Year", average_gross_rent(sfo_data))#
Average_Sale_Price = pn.Column("## Average Sale Price", average_sales_price(sfo_data))
Average_Price_by_Neighborhood = pn.Column("## Average Price By Neighborhood", average_price_by_neighborhood(sfo_data))
Top_Ten_Most_Expensive = pn.Column("##Top 10 Most Expensive Neighborhoods", top_most_expensive_neighborhoods(sfo_data))
Most_Expensive_Neighborhood_Rent_VS_Sale_Prices = pn.Column("## most expensive neighborhoods rent sales",
                                                            most_expensive_neighborhoods_rent_sales(sfo_data))

# Create the dashboar
Neighborhood_Dash = pn.Tabs(("Housing Units Per Year", Housing_Units),
                            ("Average Gross Rent in San Francisco Per Year.", Gross_Rent), 
                            ("Average Sales Price Per Year", Average_Sale_Price),
                            ("Average Prices by Neighborhood", Average_Price_by_Neighborhood),
                            ("Top 10 Most Expensive Neighborhoods", Top_Ten_Most_Expensive), 
                            ("Comparison of Rent and Sales Prices of Most Expensive Neighborhoods", Most_Expensive_Neighborhood_Rent_VS_Sale_Prices))
                            


## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
Neighborhood_Dash.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()
housing_units_per_year(sfo_data)

In [ ]:
# average_gross_rent()
average_gross_rent(sfo_data)

In [ ]:
# average_sales_price()
average_sales_price(sfo_data)

In [ ]:
# average_price_by_neighborhood()
average_price_by_neighborhood(sfo_data)

In [ ]:
# top_most_expensive_neighborhoods()
top_most_expensive_neighborhoods(sfo_data)

In [ ]:
# most_expensive_neighborhoods_rent_sales()
most_expensive_neighborhoods_rent_sales(sfo_data)

In [ ]:
#hood_P_RI = sfo_data.groupby(['year', 'neighborhood']).mean()
    #hood_p_RI_plot = hood_P_RI.hvplot.bar(groupby='neighborhood', y=['sale_price_sqr_foot', 'gross_rent'], ylabel='number of housing units', xlabel='neighborhood', rot=90)
   # this is a second way of doing it
    #return hood_p_RI_plot

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()